In [ ]:
import requests # http requests
import re # regular expressions
from bs4 import BeautifulSoup # xml parsing
import json
import tqdm
from concurrent.futures import ThreadPoolExecutor

In [ ]:
# send a HTTP GET request and store the response
occupations = {"male": [
    "police officer",
    "electrician",
    "carpenter",
    "software developer"],
"female": [
    "social worker",
    "maid",
    "secretary",
    "nurse"],
"general": [
    "professor",
    "photographer",
    "artist",
    "manager"]
}

url = 'https://knn5.laion.ai/knn-service'
for gender,occupation in occupations.items():
    
    for occ in occupation:
        data = {"text":f"A picture of a {occ}","image":None,"image_url":None,"embedding_input":None,"modality":"image","num_images":200,"indice_name":"laion5B","num_result_ids":3000,"use_mclip":False,"deduplicate":True,"use_safety_model":False,"use_violence_detector":True,"aesthetic_score":"","aesthetic_weight":"0.5"}
        response = requests.post(url, data=json.dumps(data))
        response.raise_for_status()
        dicts = json.loads(response.text)
        cnt = 0
        idx = 0
        
        with ThreadPoolExecutor(max_workers = 4) as executor:
            def func(dic):
                global idx
                imgurl = dic.get('url')
                r = requests.get(imgurl)
                try: 
                    r.raise_for_status()
                except: 
                    return
                with open(f'pics/{occ}_{idx}.jpg', 'wb') as f:
                    f.write(r.content)
                idx += 1
            list(map(func, tqdm.tqdm(dicts)))
        print(occ,'error',cnt)
'''if status_code != 200:
    raise Exception(f'Error {status_code} ({requests.status_codes._codes[status_code]})')
else:
    html_text = response.text
    print(f'Request successful')'''